# Cross-Entropy Loss
Cross-entropy / log loss is a common loss function for binary classifiers.

## A Simple Classification Problem

We'll start with 10 random points:

```x = [-2.2, -1.4, -0.8, 0.2, 0.4, 0.8, 1.2, 2.2, 2.9, 4.6]```

This is our only feature, **x**:
![x](https://cdn-images-1.medium.com/max/1200/1*233u_eZT4P8Q7uC9ES9jFw.png)

Now we'll assign some colors to our points - red and green - these are our labels:
![colors](https://cdn-images-1.medium.com/max/1200/1*zdmnljReyuOb6_h6luMaNw.png)

Our problem is pretty obvious: predict, given our feature **x**, we need to predict it's label, **red or green**.

Since this is a **binary classification**, we can also pose this problem as: **"is the point green?"** or even better, **"what is the probability of the point being green?"** Ideally, green points would have a probability of **1.0** (of being green), while **red points** would have a probability of **0.0** (of being green).

Within this context, green points belong to the **positive class** (YES, they are green), while red points belong to the **negative class** (NO, they are not green).

If we fit a model to perform this classification, **it will predict a probability of being green** to each one of our points. Given what we know about the color of the points, how can we evalute how good/bad the predicted probabilities are? By using a loss function. It should return **high values for bad predictions and low values for good predictions**.

## Loss Function: Binary Cross-Entropy / Log Loss
Mathematically, this looks like:

$$
H_p(q) = \frac{1}{N} \sum^{N}_{i=1}y_i \times log(p(y_i)) + (1-y_i) \times log(1-p(y_i))
$$
where $y$ is the label (1 for green points and 0 for red points) and $p(y)$ is the predicted probability of the point being green for all $N$ points.

This formula tells us that for each green point ($y=1$), it adds $log(p(y))$ to the loss, that is, the **log probability of it being green**. Conversely, it adds $log(1-p(y))$, that is, the **log probability of it being red** for each red point ($y=0$). Not difficult, but not intuitive.

## Visual Representation
First, let's split the points according to their classes **positive** and **negative**:
![pos neg](https://cdn-images-1.medium.com/max/1200/1*Fa-WkLN9vg_2uhOg-47XZw.png)

Next, let's train a **Logistic Regression** to classify our points. The fitted regression is a *Sigmoid Curve* representing the **probability of a point being green for any given $x$**:
![sigmoid](https://cdn-images-1.medium.com/max/1200/1*5T7AYwKH_9InxK9gc4n5hA.png)

Then, for all points belonging to the positive class, green, what are the predicted probabilities given by our classifier? These are the green bars **under** the sigmoid curve, at the $x$ coordinates corresponding to the points:
![green bars](https://cdn-images-1.medium.com/max/1200/1*MgSLK7Dz14devDqfg8g-7Q.png)

Now for the negative class. Since the green bars **under** the sigmoid curve represent the probability of a given point being green...the probability of a given point being red will be the red bars **above** the sigmoid curve:
![red bars](https://cdn-images-1.medium.com/max/1200/1*j8cBH1Pcr0CHmXPgzgKCPQ.png)

All together, we have this:
![all bars](https://cdn-images-1.medium.com/max/1200/1*JLdsBjbAz2zwgQ9m977Bsw.png)

The bars represent the **predicted probabilities** associated with the corresponding **true class** of each point.

Now that we have the predicted probabilities, time to evaluate them by computing the **binary cross-entropy / log loss**.

The probabilities are all we need, so lets get rid of the **x axis** and bring the bars together. The haning bars (at the top) don't matter anymore as well, so we're looking at something like this:
![new axis](https://cdn-images-1.medium.com/max/1200/1*72oORljVj0UjFFHLy1rJyA.png)

Since we're trying to compute a loss, we need to penalize bad predictions. If the probability associated with a **true class** is **1.0**, we need its loss to be zero. Conversely, if the probability is low, say, **0.1**, we need its loss to be **HUGE**.

It turns out, taking the **(negative) log of the probability** suits us well enough for this purpose (since the log of values between 0.0 and 1.0 is negative, we take the negative log to obtain a positive value for the loss).

The plot below gives us a clearer picture of this. As the **predicted probability of a true class gets closer to zero, the loss increases exponentially**:
![log loss](https://cdn-images-1.medium.com/max/1200/1*T8KWtAn8FkAcsg8RsjiZ6Q.png)

Now, let's take **the negative log of the probabilities** - these are the corresponding losses of each and every point. Finally, we compute the **mean of all these losses**:
![mean loss](https://cdn-images-1.medium.com/max/1200/1*i9EPio8R8j1Dd5kdaLy6pQ.png)

And that's it! The **binary cross-entropy / log loss** of this visual example is $0.3329$

### Code Implementation of the above:

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import numpy as np

x = np.array([-2.2, -1.4, -.8, .2, .4, .8, 1.2, 2.2, 2.9, 4.6])
y = np.array([0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

logr = LogisticRegression(solver='lbfgs')
logr.fit(x.reshape(-1, 1), y)

y_pred = logr.predict_proba(x.reshape(-1, 1))[:, 1].ravel()
loss = log_loss(y, y_pred)

print(f"x = {x}")
print(f"y = {y}")
print(f"p(y) = {np.round(y_pred, 2)}")
print('Log Loss / Cross Entropy = {:.4f}'.format(loss))

x = [-2.2 -1.4 -0.8  0.2  0.4  0.8  1.2  2.2  2.9  4.6]
y = [0. 0. 1. 0. 1. 1. 1. 1. 1. 1.]
p(y) = [0.19 0.33 0.47 0.7  0.74 0.81 0.86 0.94 0.97 0.99]
Log Loss / Cross Entropy = 0.3329


## The Math
### Distribution
Since $y$ represents the **classes** of our points (we have 3 red and 7 green points), this is what its distribution, $q(y)$ looks like:
![distribution](https://cdn-images-1.medium.com/max/1200/1*QTufu69NX4W9tZQD1AduQQ.png)

### Entropy
**Entropy is a measure of the uncertainty associated with a given distribution $q(y)$**.

What is all of our points were green? Well, the uncertainty of that distribution would be ZERO - there would be no doubt about the color of the point: it's always green. So entropy would be zero.

On the other hand, if we knew half of the points were green and half red, the *worst case scenario*, we would have **no edge on guessing the color of a point**: it is totally random. For that case, entropy is given by the formula below (we have two classes - red or green - hence $2$):

$$
H(q) = log(2)
$$

For every other case in between, we can compute the entropy of a distribution, $q(y)$, using the formula below, where $C$ is the number of classes:

$$
H(q) = -\sum_{c=1}^{C} q(y_c) \times log(q(y_c))
$$

If we know the **true distribution** of a random variable, we can compute its **entropy**...so why bother training a classifier in the first place?

Well...if we don't know the distribution - we can try to **approximate the true distribution** with some other distribution, say $p(y)$.

### Cross-Entropy
Let's assume our points follow this other distribution, **$p(y)$**, but we know they are **actually** coming from the **true** (unknown) distribution, **$q(y)$**

If we compute entropy like this, we are actually computing the **cross-entropy** between both distributions:

$$
H_p(q) = -\sum_{c=1}^{C} q(y_c) \times log(p(y_c))
$$

If we, miraculously, match $p(y)$ to $q(y)$ perfectly, the computed values for entropy and cross-entropy will match. Since this will ikely never happen, **cross-entropy** will have a BIGGER value than the entropy computed on the true distribution: $H_p(q) - H(q) >= 0$

The difference between the cross-entropy and entropy is called the....

#### Kullback-Leibler Divergence
The **Kullback-Leibler Divergence** is a measure of the **dissimilarity between two distributions*:

$$
D_{KL}(q||p) = H_p(q) - H(q) = \sum_{c=1}^{C} q(y_c) \times [log(q(y_c)) - log(p(y_c))]
$$

This means that, the **closer $p(y)$ gets to $q(y)$, lower the divergence, and consequently, the cross-entropy, will be**.

So we need a good $p(y)$ to use...but this is what our classifier should do - and it does! It looks for the best possible $p(y)$, which is the one that **minimizes the cross-entropy**.

### Loss Function
During its training, the classifier uses each of the $N$ points in its training set to compute the cross-entropy loss, effectively **fitting the distribution $p(y)$**. Since the probability of each point is $\frac{1}{N}$, cross-entropy is given by:

$$
q(y_i) = \frac{1}{N} \Rightarrow H_p(q) = -\frac{1}{N} \sum_{i=1}^{N} log(p(y_i))
$$

We then need to compute the cross-entropy on top of the *probabilities associated with the true class* of each point. This means using the **green bars** for the points in the **positive class** (y=1) and the **red hanging bars** for the points in the negative class (y=0), or mathematically:

$$
y_i = 1 \Rightarrow log(p(y_i))
y_i = 0 \Rightarrow log(1-p(y_i))
$$

The final step is to compute the average of all points in both classes, positive and negative:

$$
H_p(q) = - \frac{1}{N_{pos}+N_{neg}} \Bigg[\sum_{i=1}^{N_{pos}}log(p(y_i)) + \sum_{i=1}^{N_{neg}}log(1 - p(y_i))\Bigg]
$$

And that's it! The original formula for **binary cross-entropy / log loss**.